In [ ]:
!pip install uv
!uv pip install  -r requirements.txt 

In [ ]:
import snowflake
from snowflake.snowpark.context import get_active_session


session = get_active_session()

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Stage or location to check
stage_location = """snow://workspace/USER$.PUBLIC."EY-AI-and-Data-Challenge"/versions/live/terraclimate_training_soil_data/"""

# Run LIST command to get files in the stage
df_files = session.sql(f"LIST '{stage_location}'")

# Count the number of files
file_count = df_files.count()

df_files.show()

print(f"Number of files in {stage_location}: {file_count}")




In [ ]:
import snowflake
from snowflake.snowpark.context import get_active_session
session = get_active_session()

from pathlib import Path
import zipfile

folder_name = "terraclimate_training_vpd_data"

source_dir = Path(f"./{folder_name}/")      # folder containing 60 CSVs
zip_path   = Path(f"/tmp/{folder_name}.zip")

csv_files = sorted(source_dir.glob("*.csv"))  # or "**/*.csv" for recursive

if not csv_files:
    raise FileNotFoundError(f"No CSV files found in {source_dir}")

with zipfile.ZipFile(zip_path, mode="w", compression=zipfile.ZIP_DEFLATED) as zf:
    for file in csv_files:
        zf.write(file, arcname=file.name)

print(f"Created: {zip_path} ({len(csv_files)} files)")

session.sql(f"""
    PUT file:///tmp/{folder_name}.zip
    'snow://workspace/USER$.PUBLIC."EY-AI-and-Data-Challenge"/versions/live/'
    AUTO_COMPRESS=FALSE
    OVERWRITE=TRUE
""").collect()

print("File saved! Refresh the browser to see the files in the sidebar")

In [ ]:
import snowflake
from snowflake.snowpark.context import get_active_session

session = get_active_session()

import warnings
warnings.filterwarnings("ignore")

from pathlib import Path
import pandas as pd


source_data = ['terraclimate']
terraclimate_variables = ['aet', 'pet', 'q', 'ppt', 'soil', 'tmax', 'tmin', 'vpd']


for i in range(len(source_data)):
    folder_name = source_data[i]
    print(folder_name)
    source_dir = Path(f"./data/{folder_name}/")  

    csv_files = sorted(source_dir.glob("*.csv"))

    #training_df = pd.DataFrame()
    training_df = pd.read_csv('data/water_quality_training_dataset.csv')

    for col in training_df.columns:
        training_df = training_df.rename(columns={col: col.lower()})

    submission_df = pd.read_csv('data/submissions_template.csv')

    for col in submission_df.columns:
        submission_df = submission_df.rename(columns={col: col.lower()})
    
    for i in range(len(csv_files)):

    #for i in range(2):

        print(csv_files[i])

        temp_df = pd.read_csv(f"{csv_files[i]}")

        for col in temp_df.columns:
            temp_df = temp_df.rename(columns={col: col.lower()})

        
        if 'training' in str(csv_files[i]):

            training_df = training_df.merge(temp_df, on=['Latitude', 'Longitude', 'Sample Date'], how='inner')
            
        elif 'validation' in str(csv_files[i]):
            
            submission_df = submission_df.merge(temp_df, on=['Latitude', 'Longitude', 'Sample Date'], how='inner'        

            

In [ ]:
display(main_df)